In [1]:
import numpy as np
from pathlib import Path
import xarray as xr

In [2]:
data_path = Path("../xorca_test_data/GYRE/")
print(sorted(data_path.glob("*.nc")))

[PosixPath('../xorca_test_data/GYRE/GYRE_5d_00010101_00011230_grid_T.nc'), PosixPath('../xorca_test_data/GYRE/GYRE_5d_00010101_00011230_grid_U.nc'), PosixPath('../xorca_test_data/GYRE/GYRE_5d_00010101_00011230_grid_V.nc'), PosixPath('../xorca_test_data/GYRE/GYRE_5d_00010101_00011230_grid_W.nc'), PosixPath('../xorca_test_data/GYRE/mesh_mask.nc')]


In [3]:
T_files = sorted(data_path.glob("*grid_T.nc"))
U_files = sorted(data_path.glob("*grid_U.nc"))
V_files = sorted(data_path.glob("*grid_V.nc"))
W_files = sorted(data_path.glob("*grid_W.nc"))
mesh_files = sorted(data_path.glob("m*.nc"))

In [4]:
ds_T = xr.open_mfdataset(T_files, decode_times=False)
ds_U = xr.open_mfdataset(U_files, decode_times=False)
ds_V = xr.open_mfdataset(V_files, decode_times=False)
ds_W = xr.open_mfdataset(W_files, decode_times=False)
ds_mesh = xr.open_mfdataset(mesh_files)

In [5]:
ds_coords = xr.Dataset()

ds_coords.coords["lon_cc"] = ds_mesh.glamt
ds_coords.coords["lon_rc"] = ds_mesh.glamu
ds_coords.coords["lon_cr"] = ds_mesh.glamv
ds_coords.coords["lon_rr"] = ds_mesh.glamf

ds_coords.coords["lat_cc"] = ds_mesh.gphit
ds_coords.coords["lat_rc"] = ds_mesh.gphiu
ds_coords.coords["lat_cr"] = ds_mesh.gphiv
ds_coords.coords["lat_rr"] = ds_mesh.gphif

ds_coords.coords["z_c"] = ds_mesh.gdept_1d
ds_coords.coords["z_l"] = ds_mesh.gdepw_1d

ds_coords

<xarray.Dataset>
Dimensions:  (t: 1, x: 32, y: 22, z: 31)
Coordinates:
    lon_cc   (t, y, x) float32 dask.array<shape=(1, 22, 32), chunksize=(1, 22, 32)>
    lon_rc   (t, y, x) float32 dask.array<shape=(1, 22, 32), chunksize=(1, 22, 32)>
    lon_cr   (t, y, x) float32 dask.array<shape=(1, 22, 32), chunksize=(1, 22, 32)>
    lon_rr   (t, y, x) float32 dask.array<shape=(1, 22, 32), chunksize=(1, 22, 32)>
    lat_cc   (t, y, x) float32 dask.array<shape=(1, 22, 32), chunksize=(1, 22, 32)>
    lat_rc   (t, y, x) float32 dask.array<shape=(1, 22, 32), chunksize=(1, 22, 32)>
    lat_cr   (t, y, x) float32 dask.array<shape=(1, 22, 32), chunksize=(1, 22, 32)>
    lat_rr   (t, y, x) float32 dask.array<shape=(1, 22, 32), chunksize=(1, 22, 32)>
    z_c      (t, z) float64 dask.array<shape=(1, 31), chunksize=(1, 31)>
    z_l      (t, z) float64 dask.array<shape=(1, 31), chunksize=(1, 31)>
Dimensions without coordinates: t, x, y, z
Data variables:
    *empty*

In [6]:
def arr_is_matching(arr, ref, **kwargs):
    return np.allclose(arr, ref, **kwargs)

In [7]:
def detect_vertical_grid(ds, ds_coords):
    possible_grids = ["c", "l"]
    for pg in possible_grids:
        for v in filter(lambda s: s.startswith("depth"), ds.coords.keys()):
            if arr_is_matching(ds_coords[f"z_{pg}"], ds[v]):
                return pg

In [8]:
def detect_horizontal_grid(ds, ds_coords):
    possible_grids = ["cc", "cr", "rc", "rr"]
    for pg in possible_grids:
        if (arr_is_matching(ds_coords[f"lon_{pg}"], ds.nav_lon)
            and arr_is_matching(ds_coords[f"lat_{pg}"], ds.nav_lat)):
            return pg

In [9]:
for ds in [ds_T, ds_U, ds_V, ds_W]:
    print(str(ds.file_name), detect_horizontal_grid(ds, ds_coords), detect_vertical_grid(ds, ds_coords))

GYRE_5d_00010101_00011230_grid_T.nc cc c
GYRE_5d_00010101_00011230_grid_U.nc rc c
GYRE_5d_00010101_00011230_grid_V.nc cr c
GYRE_5d_00010101_00011230_grid_W.nc cc l
